<a href="https://colab.research.google.com/github/alexk2206/Data_Driven_Fantasy_Football/blob/dev/Mixed_Integer_Problem_for_Draft_Optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1  Initialisierung und Konfiguration

## 1.1 Bibliotheken installieren und laden


In [16]:
import pandas as pd
import numpy as np
import re
import random
# !pip install mip
from mip import Model, BINARY, CONTINUOUS, xsum, maximize
# !pip install rapidfuzz
from rapidfuzz import process

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 27.2 MB/s eta 0:00:00


## 1.2 Liga-Parameter und Rahmenbedingungen definieren

In [17]:
year = 2024
num_teams = 12
num_rounds = 15
num_weeks = 17
allowed_positions = {'QB', 'RB', 'WR', 'TE', 'K', 'DST'}
lineup_req = {'QB': 1, 'RB': 2, 'WR': 2, 'TE': 1, 'FLEX': 1, 'K': 1, 'DST': 1}

# 2 Datenbeschaffung und -vorverarbeitung


## 2.1 ADP, Bye Weeks und Projections laden

In [50]:
adp_url = f'https://raw.githubusercontent.com/alexk2206/Data_Driven_Fantasy_Football/refs/heads/main/pre_season_data/adp_projections_{year}.csv'
player_adp = (
    pd.read_csv(adp_url)
      .rename(columns={'player': 'Player', 'position': 'POS', 'adp': 'ADP', 'points': 'TTL'})
      .fillna({'ADP': 999})
      .loc[:, ['Player', 'POS', 'ADP', 'TTL']]
      .sort_values('ADP')
      .reset_index(drop=True)
)


bye_url = f'https://raw.githubusercontent.com/alexk2206/Data_Driven_Fantasy_Football/refs/heads/main/bye_weeks/bye_weeks_{year}.csv'
bye_weeks = pd.read_csv(bye_url)


season_projection_url = f'https://raw.githubusercontent.com/alexk2206/Data_Driven_Fantasy_Football/refs/heads/main/pre_season_data/adp_projections_{year}.csv'
season_projections = (
    pd.read_csv(season_projection_url)
      .rename(columns={'player': 'Player', 'position': 'POS', 'team': 'Team', 'points': 'TTL', 'adp': 'ADP'})
      .loc[:, ['Player', 'POS', 'Team', 'TTL', 'ADP']]
      .merge(bye_weeks[['Abbreviation', 'Bye']], left_on='Team', right_on='Abbreviation', how='left')
      .drop(columns='Abbreviation')
      .drop_duplicates(subset='Player', keep='first')
)
season_projections['avg_proj'] = season_projections['TTL'] / np.where(
    season_projections['Bye'].isna(),
    num_weeks,
    num_weeks - 1
)
week_cols = [f'Week_{w}' for w in range(1, num_weeks + 1)]
for week in week_cols:
    season_projections[week] = np.where(
        season_projections['Bye'] == week,
        0,
        season_projections['avg_proj']
    )
season_projections = season_projections.drop(columns='avg_proj')

real_projections_url = f'https://raw.githubusercontent.com/alexk2206/Data_Driven_Fantasy_Football/refs/heads/main/real_projections/real_projections_{year}.csv'
real_projections = (
    pd.read_csv(real_projections_url)
      .rename(columns={'player': 'Player', 'position': 'POS', 'points': 'Projection', 'week': 'Week'})
      .loc[:, ['Player', 'POS', 'Projection', 'Week']]
      .pivot_table(
         index=['Player', 'POS'],
         columns='Week',
         values='Projection',
         aggfunc='first'
      )
      .rename_axis(None, axis=1)
      .add_prefix('Week_')
      .reset_index()
      .fillna(0)
)

real_data_RZ_url = f'https://raw.githubusercontent.com/alexk2206/Data_Driven_Fantasy_Football/refs/heads/main/Weekly_Data/weekly_data_{year}.csv'
real_data_RZ = (
    pd.read_csv(real_data_RZ_url)
    .rename(columns={'position': 'POS', 'player_display_name': 'Player'})
    .replace({'FB': 'RB'})
    .drop(columns = ['season'], errors='ignore')
    .loc[lambda real_data_RZ: real_data_RZ['POS'].isin(allowed_positions)]
)


real_data_FP_url = f'https://raw.githubusercontent.com/alexk2206/Data_Driven_Fantasy_Football/refs/heads/main/Weekly_Data/FantasyPros_Fantasy_Football_Points_PPR_{year}.csv'
real_data_FP = (
    pd.read_csv(real_data_FP_url)
      .rename(columns={'Pos': 'POS', 'player_display_name': 'Player'})
      .replace({'-': 0, 'BYE': 0})
      .drop(columns = ['#', 'Team'], errors='ignore')
      .query("POS in ['K', 'DST']")
)

real_data_FP = real_data_FP.melt(
    id_vars=['Player', 'POS'],
    value_vars=[str(w) for w in range(1, 19)],
    var_name='week',
    value_name='fantasy_points_ppr'
)
real_data_FP = real_data_FP[~real_data_FP['fantasy_points_ppr'].isin(['BYE', '-', None])].copy()
real_data_FP['fantasy_points_ppr'] = pd.to_numeric(real_data_FP['fantasy_points_ppr'])

real_data = pd.concat([real_data_FP, real_data_RZ], ignore_index=True)

print("Positions in Fantasy Pros Data:")
print(real_data_FP['POS'].value_counts())
print("\nPositions in Row Zero Data:")
print(real_data_RZ['POS'].value_counts())
print("\nPositions in Concatonated Data")
print(real_data['POS'].value_counts())

Positions in Fantasy Pros Data:
POS
K      792
DST    576
Name: count, dtype: int64

Positions in Row Zero Data:
POS
WR    2238
RB    1480
TE    1137
QB     697
Name: count, dtype: int64

Positions in Concatonated Data
POS
WR     2238
RB     1480
TE     1137
K       792
QB      697
DST     576
Name: count, dtype: int64


In [56]:
adp_names = player_adp['Player'].unique().tolist()
adp_pos_dict = dict(zip(player_adp['Player'], player_adp['POS']))

def match_name_with_pos(name, pos, reference_list, reference_pos_dict, cutoff=85.5):
    filtered_candidates = [p for p in reference_list if reference_pos_dict.get(p) == pos]
    result = process.extractOne(name, filtered_candidates, score_cutoff=cutoff)
    return result[0] if result else None

real_data['Player'] = real_data.apply(
    lambda row: match_name_with_pos(row['Player'], row['POS'], adp_names, adp_pos_dict),
    axis=1
)
real_data = real_data.drop_duplicates(subset=['Player', 'POS', 'week'], keep='first')

real_data = real_data[real_data['Player'].notnull()].copy()

In [57]:
# filter real_data for Harrison Butker
real_data[real_data['Player'] == 'Jaguars']

,Player,POS,week,fantasy_points_ppr
59,Jaguars,DST,1,4.0
135,Jaguars,DST,2,3.0
211,Jaguars,DST,3,-2.0
287,Jaguars,DST,4,4.0
363,Jaguars,DST,5,5.0
439,Jaguars,DST,6,3.0
515,Jaguars,DST,7,9.0
591,Jaguars,DST,8,2.0
667,Jaguars,DST,9,10.0
743,Jaguars,DST,10,11.0


In [27]:
pos_counts = (
    real_data
    .drop_duplicates(subset=['Player', 'POS'])['POS']
    .value_counts()
)
pos_counts

,count
POS,
WR,136
RB,122
TE,103
QB,67
K,39
DST,32


## 2.2 Berechnung von TTL, Dropoff und VOR

In [28]:
dropoff_w = {'QB':1.0,'RB':1.0,'WR':1.0,'TE':0.9,'K':0.4,'DST':0.3}
vor_w     = {'QB':0.8,'RB':1.0,'WR':1.0,'TE':0.8,'K':0.25,'DST':0.25}

rep_ttl = (
    season_projections
      .groupby('POS')['TTL']
      .apply(lambda s: s.nlargest(lineup_req[s.name]*num_teams)
                   .iloc[-1]
             if len(s) >= lineup_req[s.name]*num_teams else 0)
)

season_projections = (
    season_projections
      .sort_values(['POS','TTL'], ascending=[True,False])
      .assign(
          dropoff=lambda df: (
              df.groupby('POS')['TTL']
                .diff(-1)
                .fillna(0)
                .mul(df['POS'].map(dropoff_w))
          ),
          VOR=lambda df: (
              (df['TTL'] - df['POS'].map(rep_ttl))
                .clip(lower=0)
                .mul(df['POS'].map(vor_w))
          )
      )
      .fillna({'dropoff': 0, 'VOR': 0, 'ADP': 999})
      .sort_values(['ADP', 'TTL'], ascending=[True, False])
      .reset_index(drop=True)
)


# 3 Modellparameter und Variablenaufbau

## 3.1 Spielerlisten und Parameter-Mapping

In [29]:
# # 1. INITIALIZATION
# players            = (season_projections[['Player', 'POS', 'ADP', 'TTL', 'dropoff', 'VOR']]
#                       .copy()
#                       .sort_values(['ADP', 'TTL'], ascending=[True, False])
#                       .reset_index(drop=True))
# players['Rank']    = players.index + 1

# players_list       = players['Player'].copy().tolist()
# positions          = {'QB', 'RB', 'WR', 'TE', 'K', 'DST', 'FLEX'}
# flex_eligible      = {'RB', 'WR', 'TE'}
# weeks              = list(range(1, 18))

# pos                = dict(zip(season_projections['Player'], season_projections['POS']))
# lineup_req         = {'QB': 1, 'RB': 2, 'WR': 2, 'TE': 1, 'K': 1, 'DST': 1, 'FLEX': 1}
# max_req            = {'QB': 2, 'RB': 999, 'WR': 999, 'TE': 2, 'K': 1, 'DST': 1}
# week_cols          = [col for col in season_projections.columns if col.startswith('Week_')]

# season_projections_dict = {
# 	row['Player']: {
# 		**{int(week.replace('Week_', '')): row[week] for week in week_cols},
# 		'dropoff': row['dropoff'],
# 		'VOR':     row['VOR']
# 	}
# 	for _, row in season_projections.iterrows()
# }

# beta               = {t: 120.0 for t in weeks}
# alpha              = 0.3
# lambda_0           = 1
# lambda_1           = 3
# lambda_2           = 4
# lambda_3           = 0.5
# lambda_4           = 50
# df_sorted          = players.sort_values('Rank').reset_index(drop=True)
# topk_pct           = 0.0025

# # 2. DRAFT ORDER SETUP

# teams              = [f'Team {i+1}' for i in range(num_teams)]
# DM_team            = 'Team 1'

# draft_order        = []
# for rnd in range(num_rounds):
# 	order = teams if rnd % 2 == 0 else teams[::-1]
# 	draft_order += order

# # 3. OPPONENT PICK FUNCTION

# def opponent_pick(roster, available, Rk, lineup_req, topk_pct=topk_pct):
# 	remaining_players = sorted(available, key=lambda p: Rk[p])
# 	topk              = max(1, int(len(remaining_players) * topk_pct))

# 	deficits = {
# 		j: lineup_req[j] - sum(1 for p in roster if pos[p] == j)
# 		for j in lineup_req if j != 'FLEX'
# 	}
# 	needed = [j for j, d in deficits.items() if d > 0]

# 	if needed:
# 		candidates = [p for p in remaining_players if pos[p] in needed]
# 		pool       = candidates[:topk] if len(candidates) >= topk else candidates
# 		if pool:
# 			return random.choice(pool)

# 	return random.choice(remaining_players[:topk])

# # 4. DRAFT INITIALIZATION

# rosters     = {tm: [] for tm in teams}
# available   = set(players_list)
# draft_log   = []




## 3.2 Optimierungsvariablen und Zielfunktion konfigurieren

In [36]:
print(season_projections[['TTL', 'VOR', 'dropoff']].describe())

              TTL         VOR     dropoff
count  555.000000  555.000000  555.000000
mean   106.715856    4.952793    2.337002
std     85.435481   17.374867    4.776833
min      1.200000    0.000000    0.000000
25%     34.550000    0.000000    0.270000
50%     82.700000    0.000000    1.000000
75%    157.500000    0.000000    2.000000
max    371.000000  162.000000   45.000000


In [37]:
# 1. INITIALIZATION
players            = (season_projections[['Player', 'POS', 'ADP', 'TTL', 'dropoff', 'VOR']]
                      .copy()
                      .sort_values(['ADP', 'TTL'], ascending=[True, False])
                      .reset_index(drop=True))
players['Rank']    = players.index + 1

players_list       = players['Player'].dropna().copy().tolist()
positions          = {'QB', 'RB', 'WR', 'TE', 'K', 'DST', 'FLEX'}
flex_eligible      = {'RB', 'WR', 'TE'}
weeks              = list(range(1, 18))

pos                = dict(zip(season_projections['Player'], season_projections['POS']))
lineup_req         = {'QB': 1, 'RB': 2, 'WR': 2, 'TE': 1, 'K': 1, 'DST': 1, 'FLEX': 1}
max_req            = {'QB': 2, 'RB': 999, 'WR': 999, 'TE': 2, 'K': 1, 'DST': 1}
week_cols          = [col for col in season_projections.columns if col.startswith('Week_')]

season_projections_dict = {
	row['Player']: {
		**{int(week.replace('Week_', '')): row[week] for week in week_cols},
		'dropoff': row['dropoff'],
		'VOR':     row['VOR']
	}
	for _, row in season_projections.iterrows()
}

beta               = {t: 120.0 for t in weeks}
alpha              = 0.3
lambda_0           = 1
lambda_1           = 1
lambda_2           = 2
lambda_3           = 5
lambda_4           = 0.25
df_sorted          = players.sort_values('Rank').reset_index(drop=True)
topk_pct           = 0.0025

# 2. DRAFT ORDER SETUP

teams              = [f'Team {i+1}' for i in range(num_teams)]
DM_team            = 'Team 6'

draft_order        = []
for rnd in range(num_rounds):
	order = teams if rnd % 2 == 0 else teams[::-1]
	draft_order += order

# 3. OPPONENT PICK FUNCTION

def opponent_pick(roster, available, Rk, lineup_req, topk_pct=topk_pct):
	remaining_players = sorted(available, key=lambda p: Rk[p])
	topk              = max(1, int(len(remaining_players) * topk_pct))

	deficits = {
		j: lineup_req[j] - sum(1 for p in roster if pos[p] == j)
		for j in lineup_req if j != 'FLEX'
	}
	needed = [j for j, d in deficits.items() if d > 0]

	if needed:
		candidates = [p for p in remaining_players if pos[p] in needed]
		pool       = candidates[:topk] if len(candidates) >= topk else candidates
		if pool:
			return random.choice(pool)

	return random.choice(remaining_players[:topk])

# 4. DRAFT INITIALIZATION

rosters     = {tm: [] for tm in teams}
available   = set(players_list)
draft_log   = []


# 5. MAIN DRAFT LOOP

for pick_idx, team in enumerate(draft_order, start=1):

    for p in available:
        ranks = df_sorted.loc[df_sorted.Player == p, 'Rank']
        if len(ranks) != 1:
            print(f'Problem bei Spieler {p}: Anzahl gefundener Ränge = {len(ranks)}')

    rank_dict = dict(zip(df_sorted['Player'], df_sorted['Rank']))
    remaining_players = sorted(available, key=lambda p: rank_dict.get(p, float('inf')))
    Rk = {p: i + 1 for i, p in enumerate(remaining_players)}
    player_vars       = set(remaining_players) | set(rosters[team])
    picks_remaining   = num_rounds - len(rosters[team])

    if team == DM_team:
        m = Model(sense=maximize, solver_name='CBC')

        y = {i: m.add_var(var_type=BINARY, name=f'y_{i}') for i in remaining_players}
        x = {(i, t): m.add_var(var_type=CONTINUOUS, name=f'x_{i}_{t}') for i in remaining_players for t in weeks}
        z = {t: m.add_var(var_type=BINARY, name=f'z_{t}') for t in weeks}

        m.objective = (
            lambda_0 * xsum(season_projections_dict[i][t] * x[i, t] for i in remaining_players for t in weeks)
            + lambda_1 * xsum(z[t] for t in weeks[:15])
            + lambda_2 * xsum(z[t] for t in weeks[15:])
            + lambda_3 * xsum(season_projections_dict[i]['dropoff'] * y[i] for i in remaining_players)
            - lambda_4 * xsum(season_projections_dict[i]['VOR'] * y[i] for i in remaining_players)
        )

        # for p in rosters[team]:
        #     m += y[p] == 1
        m += xsum(y[i] for i in remaining_players) == picks_remaining

        for pos_name, req in lineup_req.items():
            if pos_name != 'FLEX':
                already_satisfied = sum(1 for p in rosters[team] if pos[p] == pos_name)
                need = max(0, req - already_satisfied)
                m += xsum(y[i] for i in remaining_players if pos[i] == pos_name) >= need

        for t in weeks:
            for j in positions - {'FLEX'}:
                m += xsum(y[i] for i in remaining_players if pos[i] == j) <= max_req[j]
            for i in remaining_players:
                m += (
                    (x[i, t] if pos[i] in positions - {'FLEX'} else 0) +
                    (x[i, t] if pos[i] in flex_eligible else 0)
                    <= 1
                )
        for i in remaining_players:
            for t in weeks:
                m += x[i, t] <= y[i]
        for t in weeks:
            m += z[t] <= xsum(season_projections_dict[i][t] * x[i, t] for i in remaining_players) / beta[t]

        n_k = pick_idx
        for future_pick in range(pick_idx + 1, pick_idx + picks_remaining * len(teams), len(teams)):
            top_cut = int(alpha * (future_pick - n_k))
            if top_cut > 0:
                top_players = [i for i, r in Rk.items() if r <= top_cut]
                m += xsum(y[i] for i in top_players) <= ((future_pick - n_k) // len(teams))

        m.optimize()
        if m.num_solutions == 0:
            raise RuntimeError(f'No feasible solution at pick {pick_idx}. Check constraints and player pool.')

        print(f"\n--- DM Pick {pick_idx} ---")
        for i in remaining_players:
            if y[i].x is not None and y[i].x >= 0.9:
                ttl = sum(season_projections_dict[i][t] for t in weeks)
                vor = season_projections_dict[i]['VOR']
                print(f"{i:25} ({pos[i]})  TTL={ttl:6.1f}   VOR={vor:6.1f}")

        chosen = [i for i in remaining_players if y[i].x is not None and y[i].x >= 0.99 and i not in rosters[team]]
        if not chosen:
            raise RuntimeError(f'No feasible pick at {pick_idx}')
        pick = min(chosen, key=lambda i: Rk[i])

    else:
        current_round   = (pick_idx - 1) // len(teams) + 1
        dynamic_topk_pct = min(current_round * 2 * topk_pct, 1.0)
        pick = opponent_pick(
            roster    = rosters[team],
            available = available,
            Rk        = Rk,
            lineup_req= lineup_req,
            topk_pct  = dynamic_topk_pct
        )

    rosters[team].append(pick)
    available.remove(pick)
    draft_log.append({
        'Pick':    pick_idx,
        'Team':    team,
        'Player':  pick,
        'Round':   (pick_idx - 1) // len(teams) + 1,
        'POS':     pos[pick]
    })

# 6. CREATE DRAFT DATAFRAME

df_draft = pd.DataFrame(draft_log)
print(m.status)
#print(m.num_constrs, m.num_vars)
print(m)



--- DM Pick 6 ---
Justin Jefferson          (WR)  TTL= 308.0   VOR=  79.0
Derrick Henry             (RB)  TTL= 230.0   VOR=  39.0
Devon Achane              (RB)  TTL= 235.0   VOR=  44.0
Travis Kelce              (TE)  TTL= 228.0   VOR=  57.6
Josh Jacobs               (RB)  TTL= 243.0   VOR=  52.0
Nico Collins              (WR)  TTL= 246.0   VOR=  17.0
Rachaad White             (RB)  TTL= 242.0   VOR=  51.0
Cooper Kupp               (WR)  TTL= 242.0   VOR=  13.0
Michael Pittman           (WR)  TTL= 243.0   VOR=  14.0
Alvin Kamara              (RB)  TTL= 241.0   VOR=  50.0
Nathaniel Dell            (WR)  TTL= 216.0   VOR=   0.0
Kyler Murray              (QB)  TTL= 308.0   VOR=  12.8
Evan Engram               (TE)  TTL= 198.0   VOR=  33.6
Harrison Butker           (K)  TTL= 156.0   VOR=   3.0
Jets                      (DST)  TTL= 100.0   VOR=   2.1

--- DM Pick 19 ---
Derrick Henry             (RB)  TTL= 230.0   VOR=  39.0
Devon Achane              (RB)  TTL= 235.0   VOR=  44.0
Josh Jaco

# 5 Ergebnisaufbereitung

## 5.1 Draft-Log exportieren und zusammenführen

In [38]:
# Gesamtprojektion pro Team
team_ttl_proj = (
    df_draft
    .merge(season_projections[['Player','TTL']], on='Player', how='left')
    .assign(TTL=lambda df: df['TTL'].fillna(0))
    .groupby('Team')['TTL']
    .sum()
    .reset_index(name='TTL_proj')
)
print(team_ttl_proj)

# Positionen zählen
position_counts = df_draft.pivot_table(
    index='Team', columns='POS', aggfunc='size', fill_value=0
)
print(position_counts)

# Draft-Infos pro Team
result_dfs = {
    team: (
        df_draft[df_draft['Team']==team]
        .sort_values('Pick')
        .assign(Pick_Info=lambda df:
            'Round '+df['Round'].astype(str)+' Pick '+df['Pick'].astype(str))
        [['Player','Pick_Info','POS']]
    )
    for team in df_draft['Team'].unique()
}

for team, df in result_dfs.items():
    print(f'=== {team} ===')
    print(df)
    print()


       Team  TTL_proj
0    Team 1    3051.0
1   Team 10    3113.6
2   Team 11    3124.1
3   Team 12    2873.4
4    Team 2    2831.5
5    Team 3    3096.7
6    Team 4    3168.1
7    Team 5    2914.6
8    Team 6    2781.2
9    Team 7    2884.9
10   Team 8    2560.6
11   Team 9    3005.5
POS      DST  K  QB  RB  TE  WR
Team                           
Team 1     1  3   2   3   1   5
Team 10    1  1   2   4   1   6
Team 11    1  1   3   4   1   5
Team 12    2  1   1   6   1   4
Team 2     1  1   1   5   2   5
Team 3     2  1   2   4   3   3
Team 4     1  1   3   3   2   5
Team 5     1  1   1   3   2   7
Team 6     1  1   1   4   2   6
Team 7     1  1   2   6   1   4
Team 8     2  1   2   6   1   3
Team 9     1  2   1   2   2   7
=== Team 1 ===
               Player          Pick_Info  POS
0         CeeDee Lamb     Round 1 Pick 1   WR
23       Drake London    Round 2 Pick 24   WR
24         James Cook    Round 3 Pick 25   RB
47      George Kittle    Round 4 Pick 48   TE
48         Josh Allen

## 5.2 Ergebnis-Statustext ausgeben

# 6 Evaluation

In [39]:
roster_projections = (
    df_draft
      .merge(
         real_projections.drop(columns=['POS']),
         on='Player',
         how='left'
      )
      .sort_values(['Team','Pick'])
      .sort_values(by=['Team', 'Pick'])
      .reset_index(drop=True)
)
roster_projections = roster_projections
roster_projections

,Pick,Team,Player,Round,POS,Week_1,Week_2,Week_3,Week_4,Week_5,...,Week_8,Week_9,Week_10,Week_11,Week_12,Week_13,Week_14,Week_15,Week_16,Week_17
0,1,Team 1,CeeDee Lamb,1,WR,18.30,20.80,20.70,20.20,19.80,...,17.50,21.00,16.60,15.10,15.80,16.70,16.70,15.60,16.80,0.00
1,24,Team 1,Drake London,2,WR,14.50,13.50,12.40,13.50,14.30,...,17.00,17.00,16.30,14.40,0.00,15.50,14.90,14.80,14.90,14.00
2,25,Team 1,James Cook,3,RB,13.80,13.50,14.40,14.20,13.70,...,13.60,15.40,15.40,14.40,0.00,15.00,15.10,14.20,15.10,15.20
3,48,Team 1,George Kittle,4,TE,11.00,11.90,0.00,11.30,12.70,...,14.20,0.00,14.90,13.60,11.30,13.00,13.10,13.70,13.80,13.70
4,49,Team 1,Josh Allen,5,QB,24.60,22.80,22.90,22.00,21.60,...,21.90,22.20,21.30,21.40,0.00,22.10,22.80,23.20,25.80,24.70
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
176,129,Team 9,Jameson Williams,11,WR,9.57,12.10,12.00,11.20,0.00,...,0.00,0.00,9.82,8.78,11.20,11.40,11.10,11.80,11.30,11.90
177,136,Team 9,Pat Freiermuth,12,TE,7.97,8.00,7.56,8.00,8.73,...,7.81,0.00,7.89,7.37,6.18,6.75,8.33,8.42,8.96,8.25
178,153,Team 9,Stefon Diggs,13,WR,14.40,14.10,13.00,14.80,13.70,...,15.60,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
179,160,Team 9,Jakobi Meyers,14,WR,10.30,9.99,8.78,12.40,11.90,...,10.70,12.40,0.00,12.80,12.30,13.20,14.40,14.70,14.40,13.30


In [40]:
num_reg_weeks = 14
playoff_weeks = [15, 16, 17]

def create_reg_schedule(teams):
    n = len(teams)
    schedule = []
    for week in range(num_reg_weeks):
        week_matches = []
        for i in range(n//2):
            team1 = teams[i]
            team2 = teams[n-1-i]
            week_matches.append((team1, team2))
        schedule.append(week_matches)
        # Rotate teams except the first one
        teams = [teams[0]] + [teams[-1]] + teams[1:-1]
    return schedule

reg_schedule = create_reg_schedule(teams)

In [45]:
# Roster limits
lineup_req = {'QB': 1, 'RB': 2, 'WR': 2, 'TE': 1, 'K': 1, 'DST': 1}

# Für jede Woche und jedes Team die beste Aufstellung bestimmen
def get_best_lineup(team, week, roster_projections, lineup_req):
    week_col = f'Week_{week}'
    team_roster = roster_projections[roster_projections['Team'] == team]
    lineup = []
    used_players = set()

    for pos, limit in lineup_req.items():
        if pos != 'FLEX':
            candidates = team_roster[team_roster['POS'] == pos]
        else:
            # FLEX-Kandidaten: RB, WR, TE, die noch nicht verwendet wurden
            candidates = team_roster[
                (team_roster['POS'].isin(['RB', 'WR', 'TE'])) &
                (~team_roster['Player'].isin(used_players))
            ]

        starters = candidates.sort_values(week_col, ascending=False).head(limit)
        lineup.append(starters)
        used_players.update(starters['Player'])

    return pd.concat(lineup)


# Punkte aus real_data holen
def get_actual_points(lineup, week, real_data):
    merged = lineup.merge(
        real_data[real_data['week'] == week],
        left_on='Player', right_on='Player', how='left'
    )
    # Fülle fehlende Werte (z.B. bei Bye Weeks) mit 0
    merged['fantasy_points_ppr'] = merged['fantasy_points_ppr'].fillna(0)
    return merged['fantasy_points_ppr'].sum()

# Für alle Wochen und alle Matchups durchlaufen
results = []
for week_idx, matchups in enumerate(reg_schedule, 1):
    for team1, team2 in matchups:
        lineup1 = get_best_lineup(team1, week_idx, roster_projections, lineup_req)
        lineup2 = get_best_lineup(team2, week_idx, roster_projections, lineup_req)
        points1 = get_actual_points(lineup1, week_idx, real_data)
        points2 = get_actual_points(lineup2, week_idx, real_data)
        winner = team1 if points1 > points2 else team2 if points2 > points1 else 'Unentschieden'
        loser = team1 if points1 < points2 else team2 if points2 < points1 else 'Unentschieden'
        results.append({
            'Woche': week_idx,
            'Team 1': team1,
            'Team 2': team2,
            'Punkte Team 1': points1,
            'Punkte Team 2': points2,
            'Sieger': winner,
            'Verlierer': loser
        })
df_results = pd.DataFrame(results)
df_results

,Woche,Team 1,Team 2,Punkte Team 1,Punkte Team 2,Sieger,Verlierer
0,1,Team 1,Team 12,90.38,101.80,Team 12,Team 1
1,1,Team 2,Team 11,97.98,58.36,Team 2,Team 11
2,1,Team 3,Team 10,72.24,82.02,Team 10,Team 3
3,1,Team 4,Team 9,68.76,99.94,Team 9,Team 4
4,1,Team 5,Team 8,84.48,61.28,Team 5,Team 8
...,...,...,...,...,...,...,...
79,14,Team 11,Team 9,82.06,107.60,Team 9,Team 11
80,14,Team 12,Team 8,118.24,84.32,Team 12,Team 8
81,14,Team 2,Team 7,65.46,125.24,Team 7,Team 2
82,14,Team 3,Team 6,60.80,148.76,Team 6,Team 3


In [58]:
def get_lineup_and_bench(team, week, roster_projections, real_data, lineup_req):
    week_col = f'Week_{week}'
    team_roster = roster_projections[roster_projections['Team'] == team].copy()

    lineup = get_best_lineup(team, week, roster_projections, lineup_req)

    lineup_players = set(lineup['Player'])
    bench = team_roster[~team_roster['Player'].isin(lineup_players)].copy()

    week_actuals = real_data[real_data['week'] == week][['Player', 'fantasy_points_ppr']]
    week_actuals = week_actuals.rename(columns={'fantasy_points_ppr': 'Actual_Points'})

    def process_df(df):
        df['Projected_Points'] = df[week_col]
        df.drop(columns=[col for col in df.columns if col.startswith("Week_")], inplace=True)
        df = df.merge(week_actuals, on='Player', how='left')
        df['Actual_Points'] = df['Actual_Points'].fillna(0)
        return df

    lineup = process_df(lineup)
    bench = process_df(bench)

    return lineup, bench

lineup, bench = get_lineup_and_bench(DM_team, 1, roster_projections, real_data, lineup_req)

pd.set_option('display.max_rows', 30)

print("=== Lineup (Week 1) ===")
display(lineup[['Player', 'POS', 'Projected_Points', 'Actual_Points']])

print("=== Bench (Week 1) ===")
display(bench[['Player', 'POS', 'Projected_Points', 'Actual_Points']])


=== Lineup (Week 1) ===


,Player,POS,Projected_Points,Actual_Points
0,Caleb Williams,QB,18.00,7.22
1,Rachaad White,RB,16.00,16.60
2,Derrick Henry,RB,13.80,10.60
3,Justin Jefferson,WR,18.50,15.90
4,Jordan Addison,WR,11.10,6.50
5,Evan Engram,TE,12.50,1.50
6,Harrison Butker,K,9.54,0.00
7,Jaguars,DST,4.56,0.00


=== Bench (Week 1) ===


,Player,POS,Projected_Points,Actual_Points
0,Trey McBride,TE,12.10,8.0
1,Chase Brown,RB,9.19,5.3
2,Rome Odunze,WR,9.47,2.1
3,Tyler Lockett,WR,10.50,13.7
4,Adam Thielen,WR,9.82,7.9
5,Brandin Cooks,WR,9.20,14.5
6,Dylan Laube,RB,1.49,0.0
